# Experimentos utilizando a rede BiLSTM-CRF com o framework FlairNLP

Os experimentos à seguir avaliam a Influencia do uso de Embeddings no Reconhecimento de Entidades Nomeadas para o Portugues, utilizando os Corpora Multi_WikiNER, LeNER_br e PL-Corpus e os embeddings Pt-Wiki-Fasttext, Flair Embeddings e BERTimbau.

Baseado nos tutoriais do flairNLP
https://github.com/flairNLP/flair

In [ ]:
%%capture
!pip install flair transformers
#seqeval git-lfs

# Corpus Multi_WikiNER

## Vetor Estático Pt-Wiki-Fasttext


### Imports

In [ ]:
## Imports

## Corpus
from flair.datasets import NER_MULTI_WIKINER

## Embeddings
from flair.embeddings import WordEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Corpus
# 1. get the corpus
corpus = NER_MULTI_WIKINER()
print(corpus)

## Tarefa
# 2. what label do we want to predict?
label_type = 'ner'

In [ ]:
## Dicionário de rótulos
# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

### Embeddings

In [ ]:
## Embeddings
# Initialize embedding
embeddings = WordEmbeddings('pt')

### Treino

In [ ]:
## Inicializando o modelo
# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

In [ ]:
## Treinando o modelo
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/sota-ner-flair',
              learning_rate=0.1,
              mini_batch_size=32,
              #embeddings_storage_mode='gpu',
              max_epochs=100)

## Vetor de Contexto Flair Embeddings


### Imports

In [ ]:
## Imports

## Corpus
from flair.datasets import NER_MULTI_WIKINER

## Importando os Embeddings, Flair-pt
from flair.embeddings import FlairEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Corpus
# 1. get the corpus
corpus = NER_MULTI_WIKINER()
print(corpus)

## Tarefa
# 2. what label do we want to predict?
label_type = 'ner'

In [ ]:
## Dicionário de rótulos
# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

### Embeddings

In [ ]:
## Empilhando os Embeddings
from flair.embeddings import StackedEmbeddings

# init Flair embeddings
flair_embedding_forward = FlairEmbeddings('pt-forward')
flair_embedding_backward = FlairEmbeddings('pt-backward')

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
embeddings = StackedEmbeddings([
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])

### Treino

In [ ]:
## Inicializando o modelo
# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Flair_NLP/sota-ner-flair'

In [ ]:
## Treinando o modelo
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train(path,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10,
              checkpoint=True)

In [ ]:
## Continuando o treinamento (Em virtude do longo tempo de treinamento foi necessário segmentar o treinamento em etapas, por essa razão foi utilizado o drive para carregar o checkpoint doe um esquema para continuar o treinamento)
trainer = ModelTrainer(tagger, corpus)

# 8. continue training at later point. Load previously trained model checkpoint, then resume
trained_model = SequenceTagger.load(path + '/checkpoint.pt')

# resume training best model, but this time until new max-epochs
trainer.resume(trained_model,
               base_path=path + '-resume',
               max_epochs=65,
               checkpoint=True,
               )

## Vetores Estático e de Contexto concatenados (Pt-Wiki-Fastext e Flair Embeddings)

### Imports

In [ ]:
## Imports

## Corpus
from flair.datasets import NER_MULTI_WIKINER

## Embeddings
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Corpus
# 1. get the corpus
corpus = NER_MULTI_WIKINER()
print(corpus)

## Tarefa
# 2. what label do we want to predict?
label_type = 'ner'

In [ ]:
## Dicionário de rótulos
# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

### Embeddings

In [ ]:
## Stacked Embeddings
# Initialize embedding stack with 
embedding_types = [
    WordEmbeddings('pt'),
    FlairEmbeddings('pt-forward'),
    FlairEmbeddings('pt-backward')
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

### Treino

In [ ]:
## Inicializando o modelo
# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Flair_NLP/sota-ner-flair'

In [ ]:
## Treinando o modelo
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train(path,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=1,
              checkpoint=True)

In [ ]:
## Continuando o treinamento (Em virtude do longo tempo de treinamento foi necessário segmentar o treinamento em etapas, por essa razão foi utilizado o drive para carregar o checkpoint doe um esquema para continuar o treinamento)
trainer = ModelTrainer(tagger, corpus)

# 8. continue training at later point. Load previously trained model checkpoint, then resume
trained_model = SequenceTagger.load(path + '/checkpoint.pt')

# resume training best model, but this time until new max-epochs
trainer.resume(trained_model,
               base_path=path + '-resume',
               max_epochs=40,
               checkpoint=True,
               )

## Vetor de Contexto BERTimbau


### Imports

In [ ]:
## Imports

## Corpus
from flair.datasets import NER_MULTI_WIKINER

## Importando os Embeddings, BERTinbaum e Flair-pt
from flair.embeddings import TransformerWordEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Corpus
# 1. get the corpus
corpus = NER_MULTI_WIKINER() #.downsample(0.8)
print(corpus)

## Tarefa
# 2. what label do we want to predict?
label_type = 'ner'

In [ ]:
## Dicionário de rótulos
# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

### Embeddings

In [ ]:
## Apenas Bert
embeddings = TransformerWordEmbeddings('neuralmind/bert-base-portuguese-cased')

### Treino

In [ ]:
## Inicializando o modelo
# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Flair_NLP/sota-ner-flair'

In [ ]:
## Treinando o modelo
# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train(path,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=15,
              checkpoint=True)

In [ ]:
## Continuando o treinamento (Em virtude do longo tempo de treinamento foi necessário segmentar o treinamento em etapas, por essa razão foi utilizado o drive para carregar o checkpoint doe um esquema para continuar o treinamento)
trainer = ModelTrainer(tagger, corpus)

# 8. continue training at later point. Load previously trained model checkpoint, then resume
trained_model = SequenceTagger.load(path + '/checkpoint.pt')

# resume training best model, but this time until new max-epochs
trainer.resume(trained_model,
               base_path=path + '-resume',
               max_epochs=40,
               checkpoint=True,
               )

# Corpus LeNER_br

## Vetor Estático Pt-Wiki-Fasttext


### Imports

In [ ]:
## Importes
## datasets
from flair.data import Corpus
from flair.datasets import ColumnCorpus

## Embeddings
from flair.embeddings import WordEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## carregando um corpus e definindo as colunas
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

## Tarefa
label_type = 'ner'

2022-12-12 00:01:51,273 Reading data from /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig
2022-12-12 00:01:51,275 Train: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/train.txt
2022-12-12 00:01:51,279 Dev: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/dev.txt
2022-12-12 00:01:51,280 Test: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/test.txt


In [ ]:
## Dicionário de rótulos
# Make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-12-12 00:02:01,521 Computing label dictionary. Progress:


7827it [00:00, 47659.78it/s]

2022-12-12 00:02:01,722 Dictionary created for label 'ner' with 7 values: ORGANIZACAO (seen 2400 times), LEGISLACAO (seen 1920 times), PESSOA (seen 1525 times), TEMPO (seen 1334 times), JURISPRUDENCIA (seen 1104 times), LOCAL (seen 611 times)
Dictionary with 7 tags: <unk>, ORGANIZACAO, LEGISLACAO, PESSOA, TEMPO, JURISPRUDENCIA, LOCAL


### Embeddings

In [ ]:
## Embeddings
# Initialize embedding
embeddings = WordEmbeddings('pt')

2022-12-12 00:02:02,756 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpxn5w9s_0


100%|██████████| 710528528/710528528 [00:53<00:00, 13373490.70B/s]

2022-12-12 00:02:56,534 copying /tmp/tmpxn5w9s_0 to cache at /root/.flair/embeddings/pt-wiki-fasttext-300d-1M.vectors.npy


2022-12-12 00:02:58,505 removing temp file /tmp/tmpxn5w9s_0
2022-12-12 00:02:59,556 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmpzhe_duta


100%|██████████| 23541010/23541010 [00:03<00:00, 7598937.33B/s] 

2022-12-12 00:03:03,311 copying /tmp/tmpzhe_duta to cache at /root/.flair/embeddings/pt-wiki-fasttext-300d-1M
2022-12-12 00:03:03,330 removing temp file /tmp/tmpzhe_duta


### Treino

In [ ]:
## Inicializando o modelo
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

2022-12-12 00:03:06,066 SequenceTagger predicts: Dictionary with 25 tags: O, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-LEGISLACAO, B-LEGISLACAO, E-LEGISLACAO, I-LEGISLACAO, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-TEMPO, B-TEMPO, E-TEMPO, I-TEMPO, S-JURISPRUDENCIA, B-JURISPRUDENCIA, E-JURISPRUDENCIA, I-JURISPRUDENCIA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL


In [ ]:
## Treinando o modelo
# Initialize trainer
trainer = ModelTrainer(tagger, corpus)

# Start training
trainer.train('resources/taggers/sota-ner-flair',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2022-12-12 00:03:14,172 ----------------------------------------------------------------------------------------------------
2022-12-12 00:03:14,175 Model: "SequenceTagger(
  (embeddings): WordEmbeddings(
    'pt'
    (embedding): Embedding(592108, 300)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=27, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2022-12-12 00:03:14,177 ----------------------------------------------------------------------------------------------------
2022-12-12 00:03:14,179 Corpus: "Corpus: 7827 train + 1176 dev + 1389 test sentences"
2022-12-12 00:03:14,181 ----------------------------------------------------------------------------------------------------
2022-12-12 00:03:14,182 Parameters:
2022-12-12 00:03:14,185  - learning_rate: "

100%|██████████| 37/37 [00:08<00:00,  4.54it/s]

2022-12-12 00:04:01,106 Evaluating as a multi-label problem: False
2022-12-12 00:04:01,127 DEV : loss 0.2923157811164856 - f1-score (micro avg)  0.3266


2022-12-12 00:04:01,213 BAD EPOCHS (no improvement): 0
2022-12-12 00:04:01,215 saving best model
2022-12-12 00:04:03,843 ----------------------------------------------------------------------------------------------------
2022-12-12 00:04:07,577 epoch 2 - iter 24/245 - loss 0.26401509 - samples/sec: 205.98 - lr: 0.100000
2022-12-12 00:04:11,315 epoch 2 - iter 48/245 - loss 0.24095730 - samples/sec: 205.71 - lr: 0.100000
2022-12-12 00:04:14,920 epoch 2 - iter 72/245 - loss 0.23139441 - samples/sec: 213.26 - lr: 0.100000
2022-12-12 00:04:18,608 epoch 2 - iter 96/245 - loss 0.22623612 - samples/sec: 208.51 - lr: 0.100000
2022-12-12 00:04:22,780 epoch 2 - iter 120/245 - loss 0.22045064 - samples/sec: 184.26 - lr: 0.100000
2022-12-12 00:04:25,881 epoch 2 - iter 144/245 - loss 0.21562629 - samples/sec: 247.96 - lr: 0.100000
2022-12-12 00:04:30,512 epoch 2 - iter 168/245 - loss 0.21055466 - samples/sec: 166.00 - lr: 0.100000
2022-12-12 00:04:33,397 epoch 2 - iter 192/245 - loss 0.20625175 - s

100%|██████████| 37/37 [00:07<00:00,  5.11it/s]

...

2022-12-12 00:48:37,597 Evaluating as a multi-label problem: False
2022-12-12 00:48:37,618 DEV : loss 0.052935414016246796 - f1-score (micro avg)  0.8009
2022-12-12 00:48:37,699 Epoch    58: reducing learning rate of group 0 to 9.7656e-05.
2022-12-12 00:48:37,701 BAD EPOCHS (no improvement): 4
2022-12-12 00:48:37,703 ----------------------------------------------------------------------------------------------------
2022-12-12 00:48:37,704 ----------------------------------------------------------------------------------------------------
2022-12-12 00:48:37,707 learning rate too small - quitting training!
2022-12-12 00:48:37,708 ----------------------------------------------------------------------------------------------------


2022-12-12 00:48:39,984 ----------------------------------------------------------------------------------------------------
2022-12-12 00:48:39,987 loading file resources/taggers/sota-ner-flair/best-model.pt
2022-12-12 00:48:41,901 SequenceTagger predicts: Dictionary with 27 tags: O, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-LEGISLACAO, B-LEGISLACAO, E-LEGISLACAO, I-LEGISLACAO, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-TEMPO, B-TEMPO, E-TEMPO, I-TEMPO, S-JURISPRUDENCIA, B-JURISPRUDENCIA, E-JURISPRUDENCIA, I-JURISPRUDENCIA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL, <START>, <STOP>


100%|██████████| 44/44 [00:08<00:00,  5.31it/s]

2022-12-12 00:48:50,382 Evaluating as a multi-label problem: False
2022-12-12 00:48:50,402 0.8704	0.8086	0.8383	0.728
2022-12-12 00:48:50,403 
Results:
- F-score (micro) 0.8383
- F-score (macro) 0.8219
- Accuracy 0.728

By class:
                precision    recall  f1-score   support

   ORGANIZACAO     0.8529    0.7525    0.7996       501
    LEGISLACAO     0.9054    0.8862    0.8957       378
        PESSOA     0.8991    0.8798    0.8894       233
         TEMPO     0.8750    0.8385    0.8564       192
JURISPRUDENCIA     0.8377    0.6973    0.7611       185
         LOCAL     0.7143    0.7447    0.7292        47

     micro avg     0.8704    0.8086    0.8383      1536
     macro avg     0.8474    0.7998    0.8219      1536
  weighted avg     0.8695    0.8086    0.8372      1536

2022-12-12 00:48:50,405 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8383395207559906,
 'dev_score_history': [0.32655737704918036,
  0.4511825348696179,
  0.567515923566879,
  0.599316133043208,
  0.636336245857186,
  0.6696750902527077,
  0.678129713423831,
  0.7148760330578512,
  0.7014542343883661,
  0.7322274881516588,
  0.7622619734564338,
  0.7377238590410167,
  0.7336368810472397,
  0.7471763683753258,
  0.727056727056727,
  0.7724377533294731,
  0.7980714690867838,
  0.787369640787949,
  0.7912467607255974,
  0.7876598110061145,
  0.7924421883812747,
  0.789443488238669,
  0.7911392405063291,
  0.7792059411596687,
  0.7920392269974043,
  0.7996580222285552,
  0.7952981651376148,
  0.7967990854529865,
  0.7959942775393419,
  0.7990828317569504,
  0.801254633589963,
  0.8028612303290414,
  0.8001142204454598,
  0.8037865748709122,
  0.8031990859754357,
  0.8011510791366907,
  0.8006853226727585,
  0.8034433285509326,
  0.8015940791346428,
  0.8029697315819532,
  0.8029697315819532,
  0.8032082497851618,
  0.8026353480378114,
  0.8

## Vetor de Contexto Flair Embeddings


### Imports

In [ ]:
## Importes
## datasets
from flair.data import Corpus
from flair.datasets import ColumnCorpus

## Embeddings
from flair.embeddings import FlairEmbeddings, StackedEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## carregando um corpus e definindo as colunas
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

## Tarefa
label_type = 'ner'

2022-12-15 22:36:41,670 Reading data from /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig
2022-12-15 22:36:41,674 Train: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/train.txt
2022-12-15 22:36:41,675 Dev: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/dev.txt
2022-12-15 22:36:41,677 Test: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/test.txt


In [ ]:
## Dicionário de rótulos
# Make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-12-15 22:36:49,338 Computing label dictionary. Progress:


7827it [00:00, 49652.16it/s]

2022-12-15 22:36:49,548 Dictionary created for label 'ner' with 7 values: ORGANIZACAO (seen 2400 times), LEGISLACAO (seen 1920 times), PESSOA (seen 1525 times), TEMPO (seen 1334 times), JURISPRUDENCIA (seen 1104 times), LOCAL (seen 611 times)
Dictionary with 7 tags: <unk>, ORGANIZACAO, LEGISLACAO, PESSOA, TEMPO, JURISPRUDENCIA, LOCAL


### Embeddings

In [ ]:
## Empilhando os Embeddings
# init Flair embeddings
flair_embedding_forward = FlairEmbeddings('pt-forward')
flair_embedding_backward = FlairEmbeddings('pt-backward')

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
embeddings = StackedEmbeddings([
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])

2022-12-15 22:37:16,551 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-pt-forward.pt not found in cache, downloading to /tmp/tmpj6hq3w1h


100%|██████████| 72819080/72819080 [01:36<00:00, 758206.02B/s]

2022-12-15 22:39:09,224 copying /tmp/tmpj6hq3w1h to cache at /root/.flair/embeddings/lm-pt-forward.pt
2022-12-15 22:39:09,280 removing temp file /tmp/tmpj6hq3w1h


2022-12-15 22:39:28,913 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-pt-backward.pt not found in cache, downloading to /tmp/tmpfd8qtr6v


100%|██████████| 72819080/72819080 [01:41<00:00, 717468.69B/s]

2022-12-15 22:41:21,566 copying /tmp/tmpfd8qtr6v to cache at /root/.flair/embeddings/lm-pt-backward.pt
2022-12-15 22:41:21,624 removing temp file /tmp/tmpfd8qtr6v


### Treino

In [ ]:
## Inicializando o modelo
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

2022-12-15 22:41:21,854 SequenceTagger predicts: Dictionary with 25 tags: O, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-LEGISLACAO, B-LEGISLACAO, E-LEGISLACAO, I-LEGISLACAO, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-TEMPO, B-TEMPO, E-TEMPO, I-TEMPO, S-JURISPRUDENCIA, B-JURISPRUDENCIA, E-JURISPRUDENCIA, I-JURISPRUDENCIA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL


In [ ]:
## Treinando o modelo
# Initialize trainer
trainer = ModelTrainer(tagger, corpus)

# Start training
trainer.train('resources/taggers/sota-ner-flair',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2022-12-15 22:41:22,092 ----------------------------------------------------------------------------------------------------
2022-12-15 22:41:22,093 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=

100%|██████████| 37/37 [00:42<00:00,  1.16s/it]

2022-12-15 22:46:30,313 Evaluating as a multi-label problem: False
2022-12-15 22:46:30,341 DEV : loss 0.17013703286647797 - f1-score (micro avg)  0.596
2022-12-15 22:46:30,469 BAD EPOCHS (no improvement): 0
2022-12-15 22:46:30,471 saving best model


2022-12-15 22:46:30,866 ----------------------------------------------------------------------------------------------------
2022-12-15 22:46:37,990 epoch 2 - iter 24/245 - loss 0.15698277 - samples/sec: 107.91 - lr: 0.100000
2022-12-15 22:46:44,692 epoch 2 - iter 48/245 - loss 0.15709476 - samples/sec: 114.67 - lr: 0.100000
2022-12-15 22:46:49,519 epoch 2 - iter 72/245 - loss 0.14674881 - samples/sec: 159.24 - lr: 0.100000
2022-12-15 22:46:55,200 epoch 2 - iter 96/245 - loss 0.14350264 - samples/sec: 135.31 - lr: 0.100000
2022-12-15 22:47:00,922 epoch 2 - iter 120/245 - loss 0.13685348 - samples/sec: 134.32 - lr: 0.100000
2022-12-15 22:47:06,815 epoch 2 - iter 144/245 - loss 0.13138519 - samples/sec: 130.42 - lr: 0.100000
2022-12-15 22:47:13,332 epoch 2 - iter 168/245 - loss 0.12520622 - samples/sec: 117.93 - lr: 0.100000
2022-12-15 22:47:18,506 epoch 2 - iter 192/245 - loss 0.12164040 - samples/sec: 148.57 - lr: 0.100000
2022-12-15 22:47:23,814 epoch 2 - iter 216/245 - loss 0.1185606

100%|██████████| 37/37 [00:10<00:00,  3.42it/s]

...

2022-12-15 23:55:31,103 Evaluating as a multi-label problem: False
2022-12-15 23:55:31,126 DEV : loss 0.03563614562153816 - f1-score (micro avg)  0.891
2022-12-15 23:55:31,257 Epoch    60: reducing learning rate of group 0 to 9.7656e-05.
2022-12-15 23:55:31,259 BAD EPOCHS (no improvement): 4
2022-12-15 23:55:31,263 ----------------------------------------------------------------------------------------------------
2022-12-15 23:55:31,265 ----------------------------------------------------------------------------------------------------
2022-12-15 23:55:31,267 learning rate too small - quitting training!
2022-12-15 23:55:31,269 ----------------------------------------------------------------------------------------------------


2022-12-15 23:55:31,637 ----------------------------------------------------------------------------------------------------
2022-12-15 23:55:31,639 loading file resources/taggers/sota-ner-flair/best-model.pt
2022-12-15 23:55:32,304 SequenceTagger predicts: Dictionary with 27 tags: O, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-LEGISLACAO, B-LEGISLACAO, E-LEGISLACAO, I-LEGISLACAO, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-TEMPO, B-TEMPO, E-TEMPO, I-TEMPO, S-JURISPRUDENCIA, B-JURISPRUDENCIA, E-JURISPRUDENCIA, I-JURISPRUDENCIA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL, <START>, <STOP>


100%|██████████| 44/44 [00:50<00:00,  1.15s/it]

2022-12-15 23:56:23,221 Evaluating as a multi-label problem: False
2022-12-15 23:56:23,241 0.8899	0.8997	0.8948	0.8187
2022-12-15 23:56:23,242 
Results:
- F-score (micro) 0.8948
- F-score (macro) 0.8802
- Accuracy 0.8187

By class:
                precision    recall  f1-score   support

   ORGANIZACAO     0.8595    0.8423    0.8508       501
    LEGISLACAO     0.9409    0.9683    0.9544       378
        PESSOA     0.9170    0.9485    0.9325       233
JURISPRUDENCIA     0.8571    0.9081    0.8819       185
         TEMPO     0.9027    0.8698    0.8859       192
         LOCAL     0.7451    0.8085    0.7755        47

     micro avg     0.8899    0.8997    0.8948      1536
     macro avg     0.8704    0.8909    0.8802      1536
  weighted avg     0.8899    0.8997    0.8945      1536

2022-12-15 23:56:23,244 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8947879572677242,
 'dev_score_history': [0.5959595959595959,
  0.7385892116182572,
  0.7973967176004527,
  0.7867378477100309,
  0.8264417845484223,
  0.8140837556681781,
  0.8287800919664593,
  0.8443213296398894,
  0.8572987721691678,
  0.8402893711741792,
  0.8522975929978118,
  0.8730245231607631,
  0.8789740849585893,
  0.8574502420656267,
  0.8776019983347211,
  0.8681408681408682,
  0.8757814623539005,
  0.8761230601687994,
  0.8906549739654699,
  0.878550175818231,
  0.8827361563517915,
  0.8868999186330351,
  0.8852997536271558,
  0.8827738029719318,
  0.8861171366594359,
  0.8935936211163046,
  0.8882224645583425,
  0.8981481481481481,
  0.8929447017161536,
  0.8973591069970052,
  0.8885236921391401,
  0.88998088998089,
  0.8892525913802509,
  0.8905228758169934,
  0.8974707642099539,
  0.8922237380627558,
  0.8922659430122115,
  0.8933623503808488,
  0.893336989306279,
  0.8863325116406463,
  0.8900409276944066,
  0.8921729611384783,
  0.8930714675395527,
  

## Vetores Estático e de Contexto concatenados (Pt-Wiki-Fastext e Flair Embeddings)


### Imports

In [ ]:
## Importes
## datasets
from flair.data import Corpus
from flair.datasets import ColumnCorpus

## Embeddings
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## carregando um corpus e definindo as colunas
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

## Tarefa
label_type = 'ner'

2022-09-11 03:40:03,613 Reading data from /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig
2022-09-11 03:40:03,619 Train: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/train.txt
2022-09-11 03:40:03,621 Dev: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/dev.txt
2022-09-11 03:40:03,624 Test: /content/drive/MyDrive/Flair_NLP/Corpus/Lener_br/Orig/test.txt


In [ ]:
## Dicionário de rótulos
# Make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-09-11 03:40:13,242 Computing label dictionary. Progress:


7827it [00:00, 28374.72it/s]

2022-09-11 03:40:13,572 Dictionary created for label 'ner' with 7 values: ORGANIZACAO (seen 2400 times), LEGISLACAO (seen 1920 times), PESSOA (seen 1525 times), TEMPO (seen 1334 times), JURISPRUDENCIA (seen 1104 times), LOCAL (seen 611 times)
Dictionary with 7 tags: <unk>, ORGANIZACAO, LEGISLACAO, PESSOA, TEMPO, JURISPRUDENCIA, LOCAL


### Embeddings

In [ ]:
## Stacked Embeddings
# Initialize embedding stack with 
embedding_types = [
    WordEmbeddings('pt'),
    FlairEmbeddings('pt-forward'),
    FlairEmbeddings('pt-backward')
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

2022-09-11 03:40:14,711 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpqoqis1c5


100%|██████████| 710528528/710528528 [01:06<00:00, 10706795.83B/s]

2022-09-11 03:41:21,924 copying /tmp/tmpqoqis1c5 to cache at /root/.flair/embeddings/pt-wiki-fasttext-300d-1M.vectors.npy


2022-09-11 03:41:23,977 removing temp file /tmp/tmpqoqis1c5
2022-09-11 03:41:25,214 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmpdn1ngcky


100%|██████████| 23541010/23541010 [00:03<00:00, 6159174.95B/s] 

...

100%|██████████| 72819080/72819080 [00:08<00:00, 8544064.06B/s] 

2022-09-11 03:41:43,315 copying /tmp/tmpohy5wift to cache at /root/.flair/embeddings/lm-pt-forward.pt
2022-09-11 03:41:43,411 removing temp file /tmp/tmpohy5wift


2022-09-11 03:41:54,449 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-pt-backward.pt not found in cache, downloading to /tmp/tmpprh54y5r


100%|██████████| 72819080/72819080 [00:08<00:00, 8713097.57B/s] 

2022-09-11 03:42:03,885 copying /tmp/tmpprh54y5r to cache at /root/.flair/embeddings/lm-pt-backward.pt
2022-09-11 03:42:03,981 removing temp file /tmp/tmpprh54y5r


### Treino

In [ ]:
## Inicializando o modelo
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

2022-09-11 03:42:04,208 SequenceTagger predicts: Dictionary with 25 tags: O, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-LEGISLACAO, B-LEGISLACAO, E-LEGISLACAO, I-LEGISLACAO, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-TEMPO, B-TEMPO, E-TEMPO, I-TEMPO, S-JURISPRUDENCIA, B-JURISPRUDENCIA, E-JURISPRUDENCIA, I-JURISPRUDENCIA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL


In [ ]:
## Treinando o modelo
# Initialize trainer
trainer = ModelTrainer(tagger, corpus)

# Start training
trainer.train('resources/taggers/sota-ner-flair',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2022-09-11 06:45:31,812 ----------------------------------------------------------------------------------------------------
2022-09-11 06:45:31,815 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'pt'
      (embedding): Embedding(592108, 300)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4396, out_features=4396, bias=T

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there "


2022-09-11 06:45:39,867 epoch 1 - iter 24/245 - loss 0.00537267 - samples/sec: 95.97 - lr: 0.100000
2022-09-11 06:45:46,894 epoch 1 - iter 48/245 - loss 0.00634968 - samples/sec: 109.38 - lr: 0.100000
2022-09-11 06:45:53,297 epoch 1 - iter 72/245 - loss 0.00599587 - samples/sec: 120.04 - lr: 0.100000
2022-09-11 06:46:00,257 epoch 1 - iter 96/245 - loss 0.00604685 - samples/sec: 110.43 - lr: 0.100000
2022-09-11 06:46:05,258 epoch 1 - iter 120/245 - loss 0.00586165 - samples/sec: 153.76 - lr: 0.100000
2022-09-11 06:46:10,997 epoch 1 - iter 144/245 - loss 0.00646426 - samples/sec: 133.95 - lr: 0.100000
2022-09-11 06:46:17,571 epoch 1 - iter 168/245 - loss 0.00642237 - samples/sec: 116.89 - lr: 0.100000
2022-09-11 06:46:25,013 epoch 1 - iter 192/245 - loss 0.00674661 - samples/sec: 103.28 - lr: 0.100000
2022-09-11 06:46:32,381 epoch 1 - iter 216/245 - loss 0.00690267 - samples/sec: 104.32 - lr: 0.100000
2022-09-11 06:46:39,770 epoch 1 - iter 240/245 - loss 0.00699894 - samples/sec: 104.02 

100%|██████████| 37/37 [00:14<00:00,  2.54it/s]

2022-09-11 06:46:55,478 Evaluating as a multi-label problem: False
2022-09-11 06:46:55,508 DEV : loss 0.04608402028679848 - f1-score (micro avg)  0.8865


...

2022-09-11 08:00:59,277 Evaluating as a multi-label problem: False
2022-09-11 08:00:59,306 DEV : loss 0.04428378865122795 - f1-score (micro avg)  0.8966


2022-09-11 08:00:59,543 Epoch    51: reducing learning rate of group 0 to 9.7656e-05.
2022-09-11 08:00:59,545 BAD EPOCHS (no improvement): 4
2022-09-11 08:00:59,549 ----------------------------------------------------------------------------------------------------
2022-09-11 08:00:59,552 ----------------------------------------------------------------------------------------------------
2022-09-11 08:00:59,555 learning rate too small - quitting training!
2022-09-11 08:00:59,556 ----------------------------------------------------------------------------------------------------
2022-09-11 08:01:04,238 ----------------------------------------------------------------------------------------------------
2022-09-11 08:01:04,241 loading file resources/taggers/sota-ner-flair/best-model.pt
2022-09-11 08:01:06,203 SequenceTagger predicts: Dictionary with 27 tags: O, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-LEGISLACAO, B-LEGISLACAO, E-LEGISLACAO, I-LEGISLACAO, S-PESSOA, B-P

100%|██████████| 44/44 [00:29<00:00,  1.47it/s]

2022-09-11 08:01:36,609 Evaluating as a multi-label problem: False
2022-09-11 08:01:36,636 0.8976	0.9134	0.9055	0.8341
2022-09-11 08:01:36,637 
Results:
- F-score (micro) 0.9055
- F-score (macro) 0.8873
- Accuracy 0.8341

By class:
                precision    recall  f1-score   support

   ORGANIZACAO     0.8605    0.8743    0.8673       501
    LEGISLACAO     0.9479    0.9630    0.9554       378
        PESSOA     0.9174    0.9528    0.9347       233
JURISPRUDENCIA     0.8342    0.8973    0.8646       185
         TEMPO     0.9727    0.9271    0.9493       192
         LOCAL     0.7609    0.7447    0.7527        47

     micro avg     0.8976    0.9134    0.9055      1536
     macro avg     0.8823    0.8932    0.8873      1536
  weighted avg     0.8984    0.9134    0.9056      1536

2022-09-11 08:01:36,641 ----------------------------------------------------------------------------------------------------


{'test_score': 0.9054533720555019,
 'dev_score_history': [0.8864688265722843,
  0.8829026937877955,
  0.8812071330589849,
  0.8873007146783947,
  0.8839900799118214,
  0.8802797955340329,
  0.8768175582990398,
  0.8880351262349067,
  0.9005975013579577,
  0.8845303867403315,
  0.8867048867048868,
  0.8793817278498481,
  0.8889496717724289,
  0.8993178717598909,
  0.902981029810298,
  0.8981152690521715,
  0.8932676518883416,
  0.884308876064853,
  0.893467064245053,
  0.8933260393873086,
  0.8932676518883416,
  0.8965517241379312,
  0.8921139101861995,
  0.895856052344602,
  0.8950095445868557,
  0.8951239444293109,
  0.8936750272628134,
  0.8955142231947483,
  0.8945791337510215,
  0.8954979536152796,
  0.8970588235294118,
  0.8932568932568933,
  0.8954408954408954,
  0.8930817610062892,
  0.8945931185144729,
  0.8938028938028939,
  0.8943488943488944,
  0.8937448784485114,
  0.8950819672131148,
  0.8941755537325676,
  0.8950819672131148,
  0.8951392681594756,
  0.896043656207367,
  0

Quero testar o curpus Lener-br, Ulisses e o Harem, gostaria de testar uma rede treinada em um corpus diferente e também quero lidar com as entidades.

# Corpus PL-Corpus

## Vetor Estático Pt-Wiki-Fasttext


### Imports

In [ ]:
## Importes
## datasets
from flair.data import Corpus
from flair.datasets import ColumnCorpus

## Embeddings
from flair.embeddings import WordEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## carregando um corpus e definindo as colunas
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='valid.txt')

## Tarefa
label_type = 'ner'

2022-10-03 18:40:31,812 Reading data from /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria
2022-10-03 18:40:31,814 Train: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/train.txt
2022-10-03 18:40:31,816 Dev: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/valid.txt
2022-10-03 18:40:31,817 Test: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/test.txt


In [ ]:
## Dicionário de rótulos
# Make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-10-03 18:40:37,873 Computing label dictionary. Progress:


6667it [00:00, 50770.50it/s]

2022-10-03 18:40:38,055 Dictionary created for label 'ner' with 8 values: PESSOA (seen 628 times), FUNDAMENTO (seen 490 times), ORGANIZACAO (seen 435 times), DATA (seen 433 times), LOCAL (seen 369 times), PRODUTODELEI (seen 230 times), EVENTO (seen 9 times)
Dictionary with 8 tags: <unk>, PESSOA, FUNDAMENTO, ORGANIZACAO, DATA, LOCAL, PRODUTODELEI, EVENTO


### Embeddings

In [ ]:
## Embeddings
# Initialize embedding
embeddings = WordEmbeddings('pt')

2022-10-03 18:40:38,193 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpw2ulevpc


100%|██████████| 710528528/710528528 [00:18<00:00, 38731417.43B/s]

2022-10-03 18:40:56,606 copying /tmp/tmpw2ulevpc to cache at /root/.flair/embeddings/pt-wiki-fasttext-300d-1M.vectors.npy


2022-10-03 18:40:58,842 removing temp file /tmp/tmpw2ulevpc
2022-10-03 18:40:59,010 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmplmp3y9p2


100%|██████████| 23541010/23541010 [00:00<00:00, 36033178.74B/s]

2022-10-03 18:40:59,731 copying /tmp/tmplmp3y9p2 to cache at /root/.flair/embeddings/pt-wiki-fasttext-300d-1M
2022-10-03 18:40:59,760 removing temp file /tmp/tmplmp3y9p2


### Treino

In [ ]:
## Inicializando o modelo
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

2022-10-03 18:41:02,599 SequenceTagger predicts: Dictionary with 29 tags: O, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-FUNDAMENTO, B-FUNDAMENTO, E-FUNDAMENTO, I-FUNDAMENTO, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-DATA, B-DATA, E-DATA, I-DATA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL, S-PRODUTODELEI, B-PRODUTODELEI, E-PRODUTODELEI, I-PRODUTODELEI, S-EVENTO, B-EVENTO, E-EVENTO, I-EVENTO


In [ ]:
## Treinando o modelo
# Initialize trainer
trainer = ModelTrainer(tagger, corpus)
path = '/content/drive/MyDrive/Flair_NLP/sota-ner-flair'

# Start training
trainer.train(path,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=100)

2022-10-03 18:41:12,330 ----------------------------------------------------------------------------------------------------
2022-10-03 18:41:12,331 Model: "SequenceTagger(
  (embeddings): WordEmbeddings(
    'pt'
    (embedding): Embedding(592108, 300)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=31, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2022-10-03 18:41:12,335 ----------------------------------------------------------------------------------------------------
2022-10-03 18:41:12,336 Corpus: "Corpus: 6667 train + 1429 dev + 1430 test sentences"
2022-10-03 18:41:12,338 ----------------------------------------------------------------------------------------------------
2022-10-03 18:41:12,339 Parameters:
2022-10-03 18:41:12,341  - learning_rate: "

100%|██████████| 45/45 [00:06<00:00,  6.64it/s]

2022-10-03 18:41:46,276 Evaluating as a multi-label problem: False
2022-10-03 18:41:46,290 DEV : loss 0.24962784349918365 - f1-score (micro avg)  0.1874


2022-10-03 18:41:46,353 BAD EPOCHS (no improvement): 0
2022-10-03 18:41:46,357 saving best model
2022-10-03 18:41:49,663 ----------------------------------------------------------------------------------------------------
2022-10-03 18:41:52,203 epoch 2 - iter 20/209 - loss 0.24736338 - samples/sec: 254.61 - lr: 0.100000
2022-10-03 18:41:54,315 epoch 2 - iter 40/209 - loss 0.24630332 - samples/sec: 303.57 - lr: 0.100000
2022-10-03 18:41:56,747 epoch 2 - iter 60/209 - loss 0.24346197 - samples/sec: 263.62 - lr: 0.100000
2022-10-03 18:41:59,049 epoch 2 - iter 80/209 - loss 0.23695731 - samples/sec: 278.51 - lr: 0.100000
2022-10-03 18:42:01,515 epoch 2 - iter 100/209 - loss 0.23573205 - samples/sec: 259.97 - lr: 0.100000
2022-10-03 18:42:04,165 epoch 2 - iter 120/209 - loss 0.23944936 - samples/sec: 241.86 - lr: 0.100000
2022-10-03 18:42:06,665 epoch 2 - iter 140/209 - loss 0.23104741 - samples/sec: 256.42 - lr: 0.100000
2022-10-03 18:42:09,093 epoch 2 - iter 160/209 - loss 0.23120208 - s

100%|██████████| 45/45 [00:06<00:00,  6.76it/s]

...

2022-10-03 19:27:01,990 Evaluating as a multi-label problem: False
2022-10-03 19:27:02,002 DEV : loss 0.035058941692113876 - f1-score (micro avg)  0.8752


2022-10-03 19:27:02,067 Epoch    85: reducing learning rate of group 0 to 9.7656e-05.
2022-10-03 19:27:02,069 BAD EPOCHS (no improvement): 4
2022-10-03 19:27:02,073 ----------------------------------------------------------------------------------------------------
2022-10-03 19:27:02,075 ----------------------------------------------------------------------------------------------------
2022-10-03 19:27:02,077 learning rate too small - quitting training!
2022-10-03 19:27:02,078 ----------------------------------------------------------------------------------------------------
2022-10-03 19:27:04,785 ----------------------------------------------------------------------------------------------------
2022-10-03 19:27:04,788 loading file /content/drive/MyDrive/Flair_NLP/sota-ner-flair/best-model.pt
2022-10-03 19:27:06,385 SequenceTagger predicts: Dictionary with 31 tags: O, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-FUNDAMENTO, B-FUNDAMENTO, E-FUNDAMENTO, I-FUNDAMENTO, S-ORGANIZACAO, B-O

100%|██████████| 45/45 [00:07<00:00,  6.03it/s]

2022-10-03 19:27:14,029 Evaluating as a multi-label problem: False
2022-10-03 19:27:14,043 0.8694	0.8447	0.8569	0.7541
2022-10-03 19:27:14,045 
Results:
- F-score (micro) 0.8569
- F-score (macro) 0.8171
- Accuracy 0.7541

By class:
              precision    recall  f1-score   support

  FUNDAMENTO     0.9174    0.8952    0.9061       124
      PESSOA     0.8926    0.9076    0.9000       119
       LOCAL     0.8081    0.7921    0.8000       101
        DATA     0.9216    0.9592    0.9400        98
 ORGANIZACAO     0.8391    0.7766    0.8066        94
PRODUTODELEI     0.7609    0.6481    0.7000        54
      EVENTO     0.8333    0.5556    0.6667         9

   micro avg     0.8694    0.8447    0.8569       599
   macro avg     0.8533    0.7906    0.8171       599
weighted avg     0.8670    0.8447    0.8548       599

2022-10-03 19:27:14,048 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8569009314140559,
 'dev_score_history': [0.18742857142857144,
  0.3552311435523115,
  0.4476190476190477,
  0.5943097997892519,
  0.6818181818181818,
  0.666,
  0.7220630372492837,
  0.7100478468899523,
  0.7585551330798479,
  0.7529411764705883,
  0.7755491881566381,
  0.7592592592592592,
  0.8048780487804877,
  0.7892204042348413,
  0.8052434456928839,
  0.8129032258064516,
  0.7939793038570083,
  0.8203198494825964,
  0.8139754485363551,
  0.8133704735376045,
  0.8176991150442477,
  0.8129032258064516,
  0.8446866485013623,
  0.8359020852221215,
  0.8442844284428443,
  0.8464285714285714,
  0.8545618789521228,
  0.8522212148685404,
  0.8434389140271493,
  0.8594306049822064,
  0.8561085972850679,
  0.8592057761732852,
  0.8576642335766423,
  0.8669064748201438,
  0.8579234972677596,
  0.8507596067917784,
  0.8651583710407241,
  0.8606485539000875,
  0.8645739910313901,
  0.8627450980392157,
  0.8664898320070734,
  0.8729874776386404,
  0.871841155234657,
  0.8663677

## Vetor de Contexto Flair Embeddings


### Imports

In [ ]:
## Importes
## datasets
from flair.data import Corpus
from flair.datasets import ColumnCorpus

## Embeddings
from flair.embeddings import FlairEmbeddings, StackedEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## carregando um corpus e definindo as colunas
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='valid.txt')

## Tarefa
label_type = 'ner'

2022-10-04 05:29:47,877 Reading data from /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria
2022-10-04 05:29:47,880 Train: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/train.txt
2022-10-04 05:29:47,882 Dev: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/valid.txt
2022-10-04 05:29:47,884 Test: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/test.txt


In [ ]:
## Dicionário de rótulos
# Make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-10-04 05:29:52,038 Computing label dictionary. Progress:


6667it [00:00, 44590.30it/s]

2022-10-04 05:29:52,237 Dictionary created for label 'ner' with 8 values: PESSOA (seen 628 times), FUNDAMENTO (seen 490 times), ORGANIZACAO (seen 435 times), DATA (seen 433 times), LOCAL (seen 369 times), PRODUTODELEI (seen 230 times), EVENTO (seen 9 times)
Dictionary with 8 tags: <unk>, PESSOA, FUNDAMENTO, ORGANIZACAO, DATA, LOCAL, PRODUTODELEI, EVENTO


### Embeddings

In [ ]:
## Stacked Embeddings
# Initialize embedding stack with 
embedding_types = [
    FlairEmbeddings('pt-forward'),
    FlairEmbeddings('pt-backward')
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

2022-10-04 05:29:52,839 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-pt-forward.pt not found in cache, downloading to /tmp/tmpgvcqwym7


100%|██████████| 72819080/72819080 [00:03<00:00, 20669355.30B/s]

2022-10-04 05:29:56,714 copying /tmp/tmpgvcqwym7 to cache at /root/.flair/embeddings/lm-pt-forward.pt
2022-10-04 05:29:56,796 removing temp file /tmp/tmpgvcqwym7


2022-10-04 05:30:08,058 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-pt-backward.pt not found in cache, downloading to /tmp/tmp1wbxvwvz


100%|██████████| 72819080/72819080 [00:03<00:00, 20063950.90B/s]

2022-10-04 05:30:12,039 copying /tmp/tmp1wbxvwvz to cache at /root/.flair/embeddings/lm-pt-backward.pt


2022-10-04 05:30:12,120 removing temp file /tmp/tmp1wbxvwvz


### Treino

In [ ]:
## Inicializando o modelo
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

2022-10-04 05:30:12,353 SequenceTagger predicts: Dictionary with 29 tags: O, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-FUNDAMENTO, B-FUNDAMENTO, E-FUNDAMENTO, I-FUNDAMENTO, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-DATA, B-DATA, E-DATA, I-DATA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL, S-PRODUTODELEI, B-PRODUTODELEI, E-PRODUTODELEI, I-PRODUTODELEI, S-EVENTO, B-EVENTO, E-EVENTO, I-EVENTO


In [ ]:
## Treinando o modelo
# Initialize trainer
trainer = ModelTrainer(tagger, corpus)
path = '/content/drive/MyDrive/Flair_NLP/sota-ner-flair'

# Start training
trainer.train(path,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=100)

2022-10-04 05:30:12,598 ----------------------------------------------------------------------------------------------------
2022-10-04 05:30:12,602 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4096, out_features=4096, bias=True)
  (rnn): LSTM(4096, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=

100%|██████████| 45/45 [00:34<00:00,  1.32it/s]

2022-10-04 05:33:22,677 Evaluating as a multi-label problem: False
2022-10-04 05:33:22,698 DEV : loss 0.16018033027648926 - f1-score (micro avg)  0.5041
2022-10-04 05:33:22,793 BAD EPOCHS (no improvement): 0
2022-10-04 05:33:22,799 saving best model


2022-10-04 05:33:23,548 ----------------------------------------------------------------------------------------------------
2022-10-04 05:33:26,676 epoch 2 - iter 20/209 - loss 0.17530746 - samples/sec: 205.22 - lr: 0.100000
2022-10-04 05:33:30,028 epoch 2 - iter 40/209 - loss 0.17048181 - samples/sec: 191.15 - lr: 0.100000
2022-10-04 05:33:33,517 epoch 2 - iter 60/209 - loss 0.15783351 - samples/sec: 183.71 - lr: 0.100000
2022-10-04 05:33:37,382 epoch 2 - iter 80/209 - loss 0.14285898 - samples/sec: 165.81 - lr: 0.100000
2022-10-04 05:33:41,266 epoch 2 - iter 100/209 - loss 0.14534819 - samples/sec: 164.99 - lr: 0.100000
2022-10-04 05:33:44,786 epoch 2 - iter 120/209 - loss 0.14638194 - samples/sec: 182.09 - lr: 0.100000
2022-10-04 05:33:47,687 epoch 2 - iter 140/209 - loss 0.14734390 - samples/sec: 220.96 - lr: 0.100000
2022-10-04 05:33:52,696 epoch 2 - iter 160/209 - loss 0.14241373 - samples/sec: 127.89 - lr: 0.100000
2022-10-04 05:33:56,484 epoch 2 - iter 180/209 - loss 0.1428196

100%|██████████| 45/45 [00:09<00:00,  4.89it/s]


...

100%|██████████| 45/45 [00:09<00:00,  4.87it/s]


2022-10-04 06:21:13,523 Evaluating as a multi-label problem: False
2022-10-04 06:21:13,539 DEV : loss 0.038046371191740036 - f1-score (micro avg)  0.8906
2022-10-04 06:21:13,634 Epoch    60: reducing learning rate of group 0 to 9.7656e-05.
2022-10-04 06:21:13,637 BAD EPOCHS (no improvement): 4
2022-10-04 06:21:13,643 ----------------------------------------------------------------------------------------------------
2022-10-04 06:21:13,645 ----------------------------------------------------------------------------------------------------
2022-10-04 06:21:13,647 learning rate too small - quitting training!
2022-10-04 06:21:13,648 ----------------------------------------------------------------------------------------------------
2022-10-04 06:21:14,329 ----------------------------------------------------------------------------------------------------
2022-10-04 06:21:14,334 loading file /content/drive/MyDrive/Flair_NLP/sota-ner-flair/best-model.pt
2022-10-04 06:21:15,156 SequenceTagge

100%|██████████| 45/45 [00:38<00:00,  1.17it/s]

2022-10-04 06:21:53,861 Evaluating as a multi-label problem: False
2022-10-04 06:21:53,877 0.8961	0.8932	0.8946	0.8193
2022-10-04 06:21:53,878 
Results:
- F-score (micro) 0.8946
- F-score (macro) 0.856
- Accuracy 0.8193

By class:
              precision    recall  f1-score   support

  FUNDAMENTO     0.9431    0.9355    0.9393       124
      PESSOA     0.9167    0.9244    0.9205       119
       LOCAL     0.8173    0.8416    0.8293       101
        DATA     0.9600    0.9796    0.9697        98
 ORGANIZACAO     0.8710    0.8617    0.8663        94
PRODUTODELEI     0.8235    0.7778    0.8000        54
      EVENTO     0.8333    0.5556    0.6667         9

   micro avg     0.8961    0.8932    0.8946       599
   macro avg     0.8807    0.8394    0.8560       599
weighted avg     0.8957    0.8932    0.8939       599

2022-10-04 06:21:53,880 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8946488294314382,
 'dev_score_history': [0.5040916530278232,
  0.618213660245184,
  0.7527472527472527,
  0.7420000000000001,
  0.7791519434628975,
  0.7723292469352013,
  0.8042895442359249,
  0.8197945845004668,
  0.82842287694974,
  0.8449682683590208,
  0.8388746803069055,
  0.8444055944055944,
  0.8363309352517986,
  0.8506151142355008,
  0.8683522231909329,
  0.8566463944396178,
  0.8631211857018308,
  0.8612354521038496,
  0.8603256212510711,
  0.8741258741258741,
  0.8881057268722466,
  0.8726003490401397,
  0.8528174936921783,
  0.8853893263342082,
  0.8821490467937607,
  0.8825065274151437,
  0.8906386701662291,
  0.8970976253298153,
  0.8788927335640138,
  0.881118881118881,
  0.896431679721497,
  0.8888888888888888,
  0.8933566433566434,
  0.8890845070422535,
  0.8913987836663771,
  0.8935427574171029,
  0.8879159369527145,
  0.8896672504378285,
  0.8857644991212653,
  0.8921739130434783,
  0.8929824561403509,
  0.8898601398601399,
  0.8879159369527145,
  0

## Vetores Estático e de Contexto concatenados (Pt-Wiki-Fastext e Flair Embeddings)

### Imports

In [ ]:
## Importes
## datasets
from flair.data import Corpus
from flair.datasets import ColumnCorpus

## Embeddings
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

## Modelo/Treino
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

### Corpus

In [ ]:
## Montando o Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## carregando um corpus e definindo as colunas
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='valid.txt')

## Tarefa
label_type = 'ner'

2022-10-05 21:18:46,495 Reading data from /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria
2022-10-05 21:18:46,497 Train: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/train.txt
2022-10-05 21:18:46,498 Dev: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/valid.txt
2022-10-05 21:18:46,499 Test: /content/drive/MyDrive/Flair_NLP/Corpus/pl_corpus_categoria/test.txt


In [ ]:
## Dicionário de rótulos
# Make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

2022-10-05 21:18:54,409 Computing label dictionary. Progress:


6667it [00:00, 53636.67it/s]

2022-10-05 21:18:54,582 Dictionary created for label 'ner' with 8 values: PESSOA (seen 628 times), FUNDAMENTO (seen 490 times), ORGANIZACAO (seen 435 times), DATA (seen 433 times), LOCAL (seen 369 times), PRODUTODELEI (seen 230 times), EVENTO (seen 9 times)
Dictionary with 8 tags: <unk>, PESSOA, FUNDAMENTO, ORGANIZACAO, DATA, LOCAL, PRODUTODELEI, EVENTO


### Embeddings

In [ ]:
## Stacked Embeddings
# Initialize embedding stack with 
embedding_types = [
    WordEmbeddings('pt'),
    FlairEmbeddings('pt-forward'),
    FlairEmbeddings('pt-backward')
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

2022-10-05 21:18:55,258 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmp4kfa13pe


100%|██████████| 710528528/710528528 [00:53<00:00, 13262686.27B/s]

2022-10-05 21:19:49,803 copying /tmp/tmp4kfa13pe to cache at /root/.flair/embeddings/pt-wiki-fasttext-300d-1M.vectors.npy


2022-10-05 21:19:51,766 removing temp file /tmp/tmp4kfa13pe
2022-10-05 21:19:52,816 https://flair.informatik.hu-berlin.de/resources/embeddings/token/pt-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmpdxhnahfl


100%|██████████| 23541010/23541010 [00:03<00:00, 6577452.98B/s] 

...

100%|██████████| 72819080/72819080 [00:06<00:00, 10789816.41B/s]

2022-10-05 21:20:07,819 copying /tmp/tmpa5j19a4h to cache at /root/.flair/embeddings/lm-pt-forward.pt


2022-10-05 21:20:07,911 removing temp file /tmp/tmpa5j19a4h
2022-10-05 21:20:18,702 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-pt-backward.pt not found in cache, downloading to /tmp/tmpcritzwjg


100%|██████████| 72819080/72819080 [00:41<00:00, 1754470.60B/s]

2022-10-05 21:21:00,910 copying /tmp/tmpcritzwjg to cache at /root/.flair/embeddings/lm-pt-backward.pt
2022-10-05 21:21:00,979 removing temp file /tmp/tmpcritzwjg


### Treino

In [ ]:
## Inicializando o modelo
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

2022-10-05 21:21:01,190 SequenceTagger predicts: Dictionary with 29 tags: O, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-FUNDAMENTO, B-FUNDAMENTO, E-FUNDAMENTO, I-FUNDAMENTO, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-DATA, B-DATA, E-DATA, I-DATA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL, S-PRODUTODELEI, B-PRODUTODELEI, E-PRODUTODELEI, I-PRODUTODELEI, S-EVENTO, B-EVENTO, E-EVENTO, I-EVENTO


In [ ]:
## Treinando o modelo
# Initialize trainer
trainer = ModelTrainer(tagger, corpus)
path = '/content/drive/MyDrive/Flair_NLP/sota-ner-flair'

# Start training
trainer.train(path,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=40)

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there "


2022-10-05 21:21:02,612 ----------------------------------------------------------------------------------------------------
2022-10-05 21:21:02,614 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'pt'
      (embedding): Embedding(592108, 300)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.5, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4396, out_features=4396, bias=T

100%|██████████| 45/45 [00:33<00:00,  1.36it/s]


2022-10-05 21:24:10,466 Evaluating as a multi-label problem: False
2022-10-05 21:24:10,483 DEV : loss 0.1848800778388977 - f1-score (micro avg)  0.3554
2022-10-05 21:24:10,587 BAD EPOCHS (no improvement): 0
2022-10-05 21:24:10,592 saving best model
2022-10-05 21:24:14,967 ----------------------------------------------------------------------------------------------------
2022-10-05 21:24:18,778 epoch 2 - iter 20/209 - loss 0.20290499 - samples/sec: 168.59 - lr: 0.100000
2022-10-05 21:24:23,038 epoch 2 - iter 40/209 - loss 0.17775158 - samples/sec: 150.35 - lr: 0.100000
2022-10-05 21:24:26,554 epoch 2 - iter 60/209 - loss 0.17685361 - samples/sec: 182.28 - lr: 0.100000
2022-10-05 21:24:29,969 epoch 2 - iter 80/209 - loss 0.16575493 - samples/sec: 187.67 - lr: 0.100000
2022-10-05 21:24:34,706 epoch 2 - iter 100/209 - loss 0.16704786 - samples/sec: 135.23 - lr: 0.100000
2022-10-05 21:24:39,112 epoch 2 - iter 120/209 - loss 0.15990150 - samples/sec: 145.40 - lr: 0.100000
2022-10-05 21:24:4

100%|██████████| 45/45 [00:08<00:00,  5.28it/s]

...

2022-10-05 21:56:48,041 Evaluating as a multi-label problem: False


2022-10-05 21:56:48,056 DEV : loss 0.04413469880819321 - f1-score (micro avg)  0.8954
2022-10-05 21:56:48,155 Epoch    40: reducing learning rate of group 0 to 3.1250e-03.
2022-10-05 21:56:48,158 BAD EPOCHS (no improvement): 4
2022-10-05 21:56:57,847 ----------------------------------------------------------------------------------------------------
2022-10-05 21:56:57,850 loading file /content/drive/MyDrive/Flair_NLP/sota-ner-flair/best-model.pt
2022-10-05 21:57:00,436 SequenceTagger predicts: Dictionary with 31 tags: O, S-PESSOA, B-PESSOA, E-PESSOA, I-PESSOA, S-FUNDAMENTO, B-FUNDAMENTO, E-FUNDAMENTO, I-FUNDAMENTO, S-ORGANIZACAO, B-ORGANIZACAO, E-ORGANIZACAO, I-ORGANIZACAO, S-DATA, B-DATA, E-DATA, I-DATA, S-LOCAL, B-LOCAL, E-LOCAL, I-LOCAL, S-PRODUTODELEI, B-PRODUTODELEI, E-PRODUTODELEI, I-PRODUTODELEI, S-EVENTO, B-EVENTO, E-EVENTO, I-EVENTO, <START>, <STOP>


100%|██████████| 45/45 [00:38<00:00,  1.18it/s]


2022-10-05 21:57:39,697 Evaluating as a multi-label problem: False
2022-10-05 21:57:39,710 0.8842	0.9048	0.8944	0.8138
2022-10-05 21:57:39,712 
Results:
- F-score (micro) 0.8944
- F-score (macro) 0.8532
- Accuracy 0.8138

By class:
              precision    recall  f1-score   support

  FUNDAMENTO     0.9200    0.9274    0.9237       124
      PESSOA     0.9256    0.9412    0.9333       119
       LOCAL     0.8037    0.8515    0.8269       101
        DATA     0.9697    0.9796    0.9746        98
 ORGANIZACAO     0.8300    0.8830    0.8557        94
PRODUTODELEI     0.8333    0.8333    0.8333        54
      EVENTO     0.7143    0.5556    0.6250         9

   micro avg     0.8842    0.9048    0.8944       599
   macro avg     0.8567    0.8531    0.8532       599
weighted avg     0.8846    0.9048    0.8943       599

2022-10-05 21:57:39,714 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8943894389438943,
 'dev_score_history': [0.35537742150968604,
  0.6008163265306123,
  0.6264840182648402,
  0.7641996557659209,
  0.7272727272727273,
  0.8236331569664904,
  0.8326105810928013,
  0.8346186803770351,
  0.8434163701067615,
  0.8622222222222221,
  0.8579040852575489,
  0.8606921029281278,
  0.8818342151675485,
  0.8445199660152931,
  0.8637137989778535,
  0.8620988725065047,
  0.8754325259515572,
  0.8796536796536796,
  0.8806228373702423,
  0.8846487424111016,
  0.8934707903780069,
  0.8796536796536796,
  0.8859878154917319,
  0.8885017421602787,
  0.889661164205039,
  0.8913793103448276,
  0.885640584694755,
  0.89198606271777,
  0.8929188255613126,
  0.8987012987012988,
  0.897212543554007,
  0.8944636678200691,
  0.8967909800520383,
  0.8985255854293148,
  0.8977469670710572,
  0.898876404494382,
  0.8954191875540191,
  0.8931034482758621,
  0.8942807625649914,
  0.8954191875540191],
 'train_loss_history': [0.37498423234156175,
  0.1498252156393744,
 